In [70]:
# Import key libraries
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random as r

In [71]:
# List the ticker symbols for the stocks of interest
lst=['BMO.TO', 'BNS.TO', 'CM.TO', 'NA.TO','RY.TO', 'TD.TO', 'ZEB.TO','ZWB.TO']
# Define a dataframe to hold all data
dfAllStocks=pd.DataFrame()
# Determine the start and end dates
start_date = '2016-08-01'
end_date = '2019-07-31'

# Gathers stock data from listed tickers and combines them into a dataframe
for ticker in lst:
    df= yf.download(ticker, start= start_date, end= end_date)
    # Remove to '.TO' suffix from Canadian stocks
    tic= ticker.replace('.TO','')
    df['Ticker']=tic
    # Drop unwanted columns
    df.drop(columns= ["Open", "High", "Low", "Close", "Volume", "Ticker"], inplace= True)
    # Change the datetime to a date
    df.index = df.index.date
    # Store each set of ticker data in a dataframe
    dfAllStocks=pd.concat([dfAllStocks, df], axis= 1)
 
# Rename the coloumns to the ticker symbols in the list
dfAllStocks.columns = lst
# Drop the N/As
dfAllStocks.dropna(inplace=True)
# Display dataframe of stocks   
display(dfAllStocks)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,BMO.TO,BNS.TO,CM.TO,NA.TO,RY.TO,TD.TO,ZEB.TO,ZWB.TO
2016-08-02,64.920082,48.621651,35.759647,34.783405,62.392979,43.958836,18.437214,11.674096
2016-08-03,64.723793,48.584629,35.924667,34.838524,62.512001,43.943203,18.453213,11.681191
2016-08-04,64.456871,48.503178,35.994350,34.751923,62.519928,43.896263,18.413212,11.666998
2016-08-05,65.045700,49.125118,36.434414,35.058968,63.043583,44.498650,18.605265,11.773451
2016-08-08,65.548157,49.362041,36.599442,35.673069,63.368858,44.623825,18.757313,11.858617
...,...,...,...,...,...,...,...,...
2019-07-24,86.878342,59.187160,43.932129,57.061752,93.263298,67.621880,25.421846,15.217304
2019-07-25,86.913177,59.557644,43.902523,56.945850,92.783829,67.360222,25.386560,15.200814
2019-07-26,87.043762,59.397659,43.817928,57.195465,93.023552,67.368927,25.404202,15.217304
2019-07-29,87.270119,59.616596,43.779865,57.088486,93.023552,67.499756,25.421904,15.229735


In [72]:
# Create dataframe to hold Monte Carlo data
dfTotalSummary=pd.DataFrame()

for ticker in lst:
    # Create temporary dataframe
    dfTemp=dfAllStocks[[ticker]]
    # Store percent change data for a given stock
    dfTemp=dfTemp[[ticker]].pct_change().dropna()
    # Create list of percentage returns
    lst=dfTemp[ticker].to_list()

    # Choose the number of simulations
    num_simulations=500
    # The number of datapoints is the number of rows
    num_datapoints= len(dfAllStocks.index)
    # Create empty dictionary to store data
    dictSim={}

    # Fill the dictionary with lists from each simulation
    for i in range(num_simulations):
        # Create an empty list for each simulation
        dictSim[i]=[]
    
        for j in range(num_datapoints):
            # Append each new list with a random historical pct return
            dictSim[i].append(r.choice(lst))

    # Convert the dictionary to a dataframe
    dfSim=pd.DataFrame(dictSim)
    # Determine the cumlative return
    dfCumReturns=(1 + dfSim).cumprod() - 1
    
    # Display the Monte Carlo simulations -  COMMENTED OUT FOR PERFORMANCE
    '''fig, ax = plt.subplots(1,1)
    ax.plot(dfCumReturns)
    ax.set_title(f"Monte Carlo Simulation of {ticker}")
    ax.set_xlabel("Trading Days")
    ax.set_ylabel("Total Percentage Return")'''

    # Convert the last row of the cumulative total returns to a list
    LastCumReturn = dfCumReturns.iloc[-1, :].tolist()
    # Add 1 to each value to get the total return
    TotalReturn = [pctReturn + 1 for pctReturn in LastCumReturn]
    # Convert the list to a dataframe
    dfLastCum = pd.DataFrame(TotalReturn)

    # Create a list of percentiles of interest
    perc = [0.025, 0.25, 0.5, 0.75, 0.975]
    # Describe the cumulative return data
    dfFullSummary= dfCumReturns.T.describe(percentiles= perc, include= float)
    # Convert the summary to a dictionary and select the last element
    FullSummary = dfFullSummary.to_dict()
    LastSummary =FullSummary[len(dfAllStocks.index)- 1]

    # Create dataframe to hold mean and two standard deviations away from the mean and 95% CI - COMMENTED OUT FOR PERFORMANCE
    dfMC_StdMean = dfFullSummary.loc[['mean', 'std', '2.5%', '97.5%']]
    dfMC_StdMean.loc['+1std'] = dfMC_StdMean.loc['mean'] + dfMC_StdMean.loc['std']
    dfMC_StdMean.loc['+2std'] = dfMC_StdMean.loc['mean'] + 2 * dfMC_StdMean.loc['std']
    dfMC_StdMean.loc['-1std'] = dfMC_StdMean.loc['mean'] - dfMC_StdMean.loc['std']
    dfMC_StdMean.loc['-2std'] = dfMC_StdMean.loc['mean'] - 2 * dfMC_StdMean.loc['std']
    
    # Display average, double standard deviation and 95% CI of the Monte Carlo
    '''fig1 = plt.figure(figsize=(10, 5))
    plt.plot(dfMC_StdMean.loc['mean'], color= "green")
    plt.plot(dfMC_StdMean.loc['2.5%'], color= "red")
    plt.plot(dfMC_StdMean.loc['97.5%'], color= "red")
    plt.plot(dfMC_StdMean.loc['+1std'], color= "yellow")
    plt.plot(dfMC_StdMean.loc['+2std'], color= "yellow")
    plt.plot(dfMC_StdMean.loc['-1std'], color= "yellow")
    plt.plot(dfMC_StdMean.loc['-2std'], color= "yellow")
    plt.title(f"Mean, Standard Deviation and 95% CI of {ticker}")
    plt.xlabel("Total Percentage Return")
    plt.ylabel("Trading Day")

    plt.show()'''

    # Increment all value by 1 except the count and standard deviation
    for key, value in LastSummary.items():
        if ( (key != 'std' ) and ( key != 'count') ):
            LastSummary[key] = LastSummary[key] + 1

    # Convert the data back into a dataframe
    dfFullSummary = pd.DataFrame(LastSummary, index= [ticker])
    # Rename the columns to show 95% confidence interval
    dfFullSummary.rename(columns= {'2.5%': '95% CI Lower', '97.5%': '95% CI Upper'}, inplace= True)
    # Reorder the columns for clarity
    dfFullSummary= dfFullSummary[['count', 'mean', 'std', 'min', '25%' ,'50%', '75%', 'max', '95% CI Lower', '95% CI Upper']]
    dfFullSummary= dfFullSummary.drop('count', axis = 1)
    
    dfTotalSummary = pd.concat([dfTotalSummary, dfFullSummary], axis= 0)
    
    # Get the 95% confidence interval
    CI_95low= LastSummary['2.5%']
    CI_95up= LastSummary['97.5%']

    # Get the mean and standard deviation
    mean= LastSummary['mean']
    stdDev= LastSummary['std']
    
    # Display histogram of the data - COMMENTED OUT FOR PERFORMANCE
    '''"fig, ax = plt.subplots(1,1)
    ax.hist(dfLastCum)
    ax.set_title(f"Distribution of Cumulative Returns of {ticker}")
    ax.set_xlabel("Total Percentage Return")
    ax.set_ylabel("Occurance")
    ax.vlines(CI_95low,0, 125,"red","solid")
    ax.vlines(CI_95up, 0, 125,"red","solid")
    ax.vlines(mean, 0, 125,"green","solid")
    ax.vlines(mean-stdDev, 0, 125,"yellow","solid")
    ax.vlines(mean+stdDev, 0, 125,"yellow","solid")
    ax.vlines(mean-2*stdDev, 0, 125,"yellow","solid")
    ax.vlines(mean+ 2*stdDev, 0, 125,"yellow","solid")'''

plt.show()

# Display dataframe
display(dfTotalSummary)

,mean,std,min,25%,50%,75%,max,95% CI Lower,95% CI Upper
BMO.TO,1.352973,0.260224,0.822327,1.159705,1.335377,1.534663,2.124608,0.885866,1.941685
BNS.TO,1.243712,0.231964,0.712751,1.079388,1.233585,1.370448,2.047067,0.825049,1.793110
CM.TO,1.249855,0.247366,0.647389,1.087761,1.228756,1.406716,2.092239,0.830256,1.798795
NA.TO,1.675114,0.350068,0.796003,1.441473,1.642688,1.886892,2.935392,1.069218,2.489396
RY.TO,1.524036,0.277942,0.866791,1.326615,1.501765,1.685030,2.836060,1.055198,2.102154
TD.TO,1.573436,0.309989,0.877007,1.346116,1.546107,1.737437,2.715937,1.070715,2.272483
ZEB.TO,1.398856,0.225423,0.801970,1.241909,1.392356,1.530952,2.161245,1.021722,1.894846
ZWB.TO,1.304172,0.187813,0.842682,1.171048,1.288929,1.414068,2.205544,0.966471,1.703368
